In [1]:
import gdown
FILE_ID = '1sNPUBO-vGky4nvJzIRtR6I1U4mbrOIuG'
url = f"https://drive.google.com/uc?id={FILE_ID}"

output = 'stunting_dataset.csv'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1sNPUBO-vGky4nvJzIRtR6I1U4mbrOIuG
To: /content/stunting_dataset.csv
100%|██████████| 2.89M/2.89M [00:00<00:00, 53.6MB/s]


'stunting_dataset.csv'

In [3]:
import pandas as pd

# Baca file CSV
# Assuming the dataset downloaded in the first cell is located here or in the specified drive path
# Using the downloaded file from the first cell for simplicity as it's in the local environment first.
df = pd.read_csv("stunting_dataset.csv")

# Ubah jenis kelamin menjadi numerik
df["JK_Code"] = df["Jenis Kelamin"].map({"Laki-laki": 1, "Perempuan": 2})

# Contoh WHO Height-for-Age Z-score lookup
who_haz_lookup = {
    (0, 1): (49.9, 1.9),
    (0, 2): (49.1, 1.8),
    (6, 1): (67.6, 2.6),
    (6, 2): (65.7, 2.5),
    (12, 1): (76.1, 2.9),
    (12, 2): (74.0, 2.8),
    (24, 1): (87.1, 3.1),
    (24, 2): (85.7, 3.0),
    (36, 1): (95.0, 3.3), # Added more data points for better lookup
    (36, 2): (93.9, 3.2),
    (48, 1): (102.8, 3.5),
    (48, 2): (101.9, 3.4),
    (59, 1): (109.7, 3.6),
    (59, 2): (108.7, 3.5),
}

# Fungsi hitung HAZ
def calculate_haz(row):
    umur = row["Umur (bulan)"]
    jk = row["JK_Code"]
    tinggi = row["Tinggi Badan (cm)"]

    # Find the closest age group for the given gender
    closest_key = None
    min_diff = float('inf')
    for (lookup_age, lookup_jk) in who_haz_lookup.keys():
        if lookup_jk == jk:
            diff = abs(lookup_age - umur)
            if diff < min_diff:
                min_diff = diff
                closest_key = (lookup_age, lookup_jk)
            elif diff == min_diff and lookup_age < umur: # Prefer the closest age less than current if tie
                 closest_key = (lookup_age, lookup_jk)
            elif diff == min_diff and lookup_age > umur and closest_key and closest_key[0] > umur: # If closest so far is also > umur, take this one if closer
                 closest_key = (lookup_age, lookup_jk)


    if closest_key in who_haz_lookup:
        median, sd = who_haz_lookup[closest_key]
        # Handle potential division by zero if SD is 0
        if sd != 0:
            return (tinggi - median) / sd
        else:
            return None # Return None if SD is 0 to avoid error
    return None

df["HAZ"] = df.apply(calculate_haz, axis=1)

# Klasifikasi stunting
def classify_stunting(haz):
    if pd.isna(haz):
        return "Unknown"
    elif haz < -3:
        return "Severely Stunted"
    elif haz < -2:
        return "Stunted"
    else:
        return "Normal"

df["Stunting_Status"] = df["HAZ"].apply(classify_stunting)

# Simpan ke file baru (Optional, but good practice if needed later)
# df.to_csv("stunting_dataset_with_status.csv", index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
import pandas as pd

# Baca file CSV
df = pd.read_csv("/content/stunting_dataset.csv")

# Ubah jenis kelamin menjadi numerik
df["JK_Code"] = df["Jenis Kelamin"].map({"Laki-laki": 1, "Perempuan": 2})

# Contoh WHO Height-for-Age Z-score lookup
who_haz_lookup = {
    (0, 1): (49.9, 1.9),
    (0, 2): (49.1, 1.8),
    (6, 1): (67.6, 2.6),
    (6, 2): (65.7, 2.5),
    (12, 1): (76.1, 2.9),
    (12, 2): (74.0, 2.8),
    (24, 1): (87.1, 3.1),
    (24, 2): (85.7, 3.0),
}

# Fungsi hitung HAZ
def calculate_haz(row):
    umur = row["Umur (bulan)"]
    jk = row["JK_Code"]
    tinggi = row["Tinggi Badan (cm)"]
    closest_key = min(who_haz_lookup.keys(), key=lambda x: abs(x[0] - umur) if x[1] == jk else float('inf'))
    if closest_key in who_haz_lookup:
        median, sd = who_haz_lookup[closest_key]
        return (tinggi - median) / sd
    return None

df["HAZ"] = df.apply(calculate_haz, axis=1)

# Klasifikasi stunting
def classify_stunting(haz):
    if pd.isna(haz):
        return "Unknown"
    elif haz < -3:
        return "Severely Stunted"
    elif haz < -2:
        return "Stunted"
    else:
        return "Normal"

df["Stunting_Status"] = df["HAZ"].apply(classify_stunting)

# Simpan ke file baru
df.to_csv("stunting_dataset_with_status.csv", index=False)
df.head()

,Id,Jenis Kelamin,Umur (bulan),Tinggi Badan (cm),Berat Badan (kg),JK_Code,HAZ,Stunting_Status
0,0,Laki-laki,19,91.6,13.3,1,1.451613,Normal
1,1,Laki-laki,20,77.7,8.5,1,-3.032258,Severely Stunted
2,2,Laki-laki,10,79.0,10.3,1,1.000000,Normal
3,3,Perempuan,2,50.3,8.3,2,0.666667,Normal
4,4,Perempuan,5,56.4,10.9,2,-3.720000,Severely Stunted


In [ ]:
# Load the uploaded dataset
file_path = '/content/stunting_dataset_with_status.csv'
df = pd.read_csv(file_path)

# Display basic info and first few rows
df.info(), df.head()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# 1. Load dataset
df = pd.read_csv("stunting_dataset_with_status.csv")

# 2. Encode jenis kelamin
df["JK_Code"] = df["Jenis Kelamin"].map({"Laki-laki": 1, "Perempuan": 2})

# 3. Drop baris dengan nilai null (jika ada)
df = df.dropna(subset=["Umur (bulan)", "Tinggi Badan (cm)", "Berat Badan (kg)", "JK_Code", "Stunting_Status"])

# 4. Label encode target
le = LabelEncoder()
df["Stunting_Label"] = le.fit_transform(df["Stunting_Status"])

# 5. Pilih fitur dan target
X = df[["Umur (bulan)", "Tinggi Badan (cm)", "Berat Badan (kg)", "JK_Code"]]
y = df["Stunting_Label"]

# 6. Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 7. Normalisasi fitur
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 8. Model klasifikasi
model = RandomForestClassifier(random_state=42)
model.fit(X_train_scaled, y_train)

# 9. Evaluasi model
y_pred = model.predict(X_test_scaled)
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# 10. Membuat DataFrame hasil prediksi
df_prediksi = X_test.copy()
df_prediksi["Actual_Label"] = le.inverse_transform(y_test)
df_prediksi["Predicted_Label"] = le.inverse_transform(y_pred)

# 11. Tampilkan 10 hasil prediksi pertama
print(df_prediksi.head(10))

In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load dataset
df = pd.read_csv("stunting_dataset_with_status.csv")
df["JK_Code"] = df["Jenis Kelamin"].map({"Laki-laki": 1, "Perempuan": 2})
df = df.dropna(subset=["Umur (bulan)", "Tinggi Badan (cm)", "Berat Badan (kg)", "JK_Code", "Stunting_Status"])

# Encode label
le = LabelEncoder()
df["Stunting_Label"] = le.fit_transform(df["Stunting_Status"])

# Fitur dan target
X = df[["Umur (bulan)", "Tinggi Badan (cm)", "Berat Badan (kg)", "JK_Code"]]
y = df["Stunting_Label"]

# Normalisasi
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Model TensorFlow
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(4,)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 kelas: Normal, Stunted, Severely Stunted
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Training
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_split=0.1)

# Evaluation
model.evaluate(X_test, y_test)

In [ ]:
def predict_stunting(model, scaler, input_data):
    """
    input_data: dict seperti
        {
            "Umur (bulan)": 24,
            "Tinggi Badan (cm)": 85.0,
            "Berat Badan (kg)": 12.5,
            "JK_Code": 1
        }
    """
    df_input = pd.DataFrame([input_data])
    scaled_input = scaler.transform(df_input)
    pred_probs = model.predict(scaled_input)
    pred_class = tf.argmax(pred_probs, axis=1).numpy()[0]
    return le.inverse_transform([pred_class])[0]

In [ ]:
save_path = '/content/drive/MyDrive/model'
tf.saved_model.save(model, save_path)

In [ ]:
!pip install tensorflowjs

In [ ]:
!tensorflowjs_converter \
  --input_format=tf_saved_model \
  --output_format=tfjs_graph_model \
  "/content/drive/MyDrive/model" \
  "/content/drive/MyDrive/model/tfjs"